Zestimate Home занимаются оценкой стоимости недвижимости.
«Zestimates» - это оценочные значения стоимости дома, основанные на 7,5 миллионах статистических моделей и моделей машинного обучения, которые анализируют сотни точек данных по каждому объекту недвижимости. 

Есть данные об ошибке их прогноза

𝑙𝑜𝑔𝑒𝑟𝑟𝑜𝑟 = 𝑙𝑜𝑔 (𝑍𝑒𝑠𝑡𝑖𝑚𝑎𝑡𝑒) −𝑙𝑜𝑔 (𝑆𝑎𝑙𝑒𝑃𝑟𝑖𝑐𝑒)

Мы хотим спрогнозировать logerror.



<h1> Будем проверять гипотезу о связи плотности участков </h1>

Высокая плотность участков указывает на большое количество точек данных в его географическом регионе. Эта `` плотность '' вокруг участка может быть связана с прогнозируемой нами логической ошибкой, поскольку производительность модели (т.е. логарифмическая ошибка) обычно зависит от количества репрезентативных данных, на которых она была обучена. Поэтому проверим следующую гипотезу:

<h3> Гипотеза: более высокая плотность участков связана с меньшей абсолютной логической ошибкой. </h3>
<p> </p>
<p> </p>


Давайте начнем!


<h1> 1. Импорт </h1>
Сначала импортируем нужные нам пакеты. Затем мы импортируем данные. Поскольку мы хотим знать взаимосвязь между логарифмической ошибкой и плотностью участков, для анализа нам нужны только строки из обучающего набора, содержащие координаты x, y. К остальным данным вернемся позже.

In [4]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

from sklearn.neighbors import KernelDensity
from scipy.stats.stats import pearsonr
from scipy.stats import rankdata


In [6]:
train = pd.read_csv('./data/train_2016_v2.csv')
props = pd.read_csv('./data/properties_2016.csv',low_memory=False)
train_ = train
train_.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [11]:
train = train_[['parcelid','longitude','latitude','logerror']]
train.dropna(inplace=True) 
display(train.head())
print("shape = ",train.shape)

KeyError: "['longitude', 'latitude'] not in index"

<h1> 2. Восстановление распределения</h1>

«Оценка ядерной плотности» или KDE. Он оценивает функцию плотности вероятности по заданным точкам данных. В нашем случае эти точки данных являются географическими координатами участков.



Будем использовать гауссовское ядро.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2a/Kernel_density.svg/250px-Kernel_density.svg.png)

Для оптимизации  производительности  функции я уже задала некоторые параметры для KDTree

In [9]:
def get_pde(train,bw):
    x = train['longitude'].values
    y = train['latitude'].values
    xy = np.vstack([x,y])
    X = np.transpose(xy)
    #tree = KDTree(X,leaf_size = 20 )     
    #parcelDensity = tree.kernel_density(X, h=bw,kernel='gaussian',rtol=0.00001)
    kde = KernelDensity(kernel='gaussian', bandwidth=bw,rtol=0.00001).fit(X)
    parcelDensity = kde.score_samples(X)
    return parcelDensity

In [10]:
parcelDensity30000 = get_pde(train,30000)
parcelDensity1000 = get_pde(train,1000)
parcelDensity300 = get_pde(train,300)


KeyError: 'longitude'

<h3> 2.1 Визуализация плотности участков с шириной окна 30 000 </h3>
Давайте визуализируем результаты нашего KDE в виде диаграммы разброса. Мы используем долготу и широту наших участков как координаты x, y и раскрашиваем эти точки по их плотности. На графике четко показаны наиболее плотные области ярко-желтым цветом, более полуцентральные участки - оранжевым, периферийные участки - фиолетовым, а относительно изолированные места - черным.

In [ ]:
plt.figure(figsize=(14,14))
plt.axis("off")
plt.title("Gaussian Density Estimate с окном 30,000")
plt.scatter(x=train['longitude'].values, y=train['latitude'].values, c=parcelDensity30000,cmap='inferno', s=1)


Ярко-желтая плотная область на карте почти идеально сочетается с тем, что Google Maps определяет как центр Лос-Анджелеса. Это означает, что функция плотности участков теперь содержит информацию о центральности определенного участка. 
https://www.google.nl/maps/place/Downtown,+Los+Angeles,+CA,+USA/@33.9761623,-118.5305526,10z/data=!4m5!3m4!1s0x80c2c634253dfd01:0x26fe52df19a5a920!8m2!3d34.040713!4d-118.2467693 

<h3> 2.2 Визуализация плотности участков с шириной окна 1000 и 300 </h3>
В KDE с меньшим размером окна локальные колебания относительно сильны и не могут вызвать выбросы, которые превосходят «нормальные» колебания в KDE на изображении графика. Чтобы улучшить видимость умеренных колебаний, оценим KDE для этих графиков по их процентилям.

In [ ]:
rankScaled30000 = 100*rankdata(parcelDensity30000)/len(parcelDensity30000)
rankScaled1000 = 100*rankdata(parcelDensity1000)/len(parcelDensity1000)
rankScaled300 = 100*rankdata(parcelDensity300)/len(parcelDensity300)

In [ ]:
fig = plt.figure(figsize=(15,15))

ax1 = fig.add_subplot(221)
ax1.set_title('ширина окна = 1,000')
ax1.set_axis_off()
ax1.scatter(train['longitude'].values, train['latitude'].values, c=parcelDensity1000,cmap='inferno', s=1)

ax2 = fig.add_subplot(222)
ax2.set_title('ширина окна = 300')
ax2.set_axis_off()
ax2.scatter(train['longitude'].values, train['latitude'].values, c=parcelDensity300,cmap='inferno', s=1)


ax3 = fig.add_subplot(223)
ax3.set_title('ширина окна = 1,000')
ax3.set_axis_off()
ax3.scatter(train['longitude'].values, train['latitude'].values, c=rankScaled1000,cmap='inferno', s=1)

ax4 = fig.add_subplot(224)
ax4.set_title('ширина окна = 300')
ax4.set_axis_off()
ax4.scatter(train['longitude'].values, train['latitude'].values, c=rankScaled300,cmap='inferno', s=1)



<h1> 3. Результаты </h1>
Мы хотим проверить, связаны ли более высокие значения KDE с более низкими абсолютными ошибками. Поэтому мы хотим сравнить KDE с абсолютной ошибкой logerror. Измерим корреляцию Пирсона для 3 размеров окна.

<h3> 3.1 Результат окна 30 000 </h3>
подробно рассмотрим результаты для окна 30 000.

In [ ]:
abs_logerrors = np.abs(train['logerror'].values)

In [ ]:
corrCoef_30000, _ = pearsonr(parcelDensity30000,abs_logerrors)

print("Результат для окна 30,000:", corrCoef_30000)

нет значимой отрицательной корреляции между функцией плотности и абсолютной логической ошибкой с окном 30 000


<h3> 3.2 Результаты для окон 1000 и 300 </h3>


In [ ]:
corrCoef_1000, _ = pearsonr(parcelDensity1000,abs_logerrors)
corrCoef_300, _ = pearsonr(parcelDensity300,abs_logerrors)

print("для окна 1,000, Correlation Coefficient: ",corrCoef_1000)
print()
print("для окна 500, Correlation Coefficient: ",corrCoef_300)




для 1,000 и 300 существует значительная отрицательная корреляция между KDE и абсолютной логической ошибкой.


<h1> 4. Заключение </h1>
Результат показывает, что более высокая плотность связана с меньшими абсолютными ошибками. Эта гипотеза подтвердилась для окон 1000 и 300. Неожиданно противоположная зависимость наблюдалась для  30 000. В целом, это означает, что плотность участков может быть полезным признаком для прогнозирования логической ошибки Zestimate и ее добавление может значительно улучшить результаты предсказаний, и на примере Zestimate, поможет им делать более точные прогнозы.


In [ ]:
cat_features = [i for i in train_.columns if train_[i].dtype !='float64']
cat_features

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
cbr = CatBoostRegressor(cat_features=cat_features, iterations=200, random_state=42)

X = train_.drop(['logerror'], axis = 1)
X[cat_features] = X[cat_features].fillna("")
y = train_['logerror']*100
scores = -np.mean(cross_val_score(cbr, X, y, cv=3, scoring='neg_mean_absolute_error'))
scores

In [ ]:
scores

In [ ]:
train_2 = pd.concat([train_, 
                     pd.DataFrame([parcelDensity1000,
                                   parcelDensity300]).T], 
                    axis = 1)

In [ ]:
cbr = CatBoostRegressor(cat_features=cat_features, iterations=200, random_state=42)

X = train_2.drop(['logerror'], axis = 1)
X[cat_features] = X[cat_features].fillna("")
y = train_['logerror'] * 100
scores = -np.mean(cross_val_score(cbr, X, y, cv=3, scoring='neg_mean_absolute_error'))
scores

In [ ]:
scores